In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

def forecast_aqi(file_path, seq_length=24, future_steps=8):
    # Load and preprocess the dataset
    df = pd.read_csv(file_path)
    df.set_index('Date_Start Time', inplace=True)
    df.drop(['Sr.NO', 'End Time'], axis=1, inplace=True)

    # Ensure AQI columns are float
    aqi_columns = ['AQI_PM2.5', 'AQI_PM10', 'AQI_NO2', 'AQI_CO', 'AQI_SO2']
    for col in aqi_columns:
        df[col] = df[col].astype(float)

    # Normalize the data using MinMaxScaler
    scalers = {}
    scaled_data = {}
    for col in aqi_columns:
        scaler = MinMaxScaler()
        scaled_data[col] = scaler.fit_transform(df[[col]])
        scalers[col] = scaler

    # Function to forecast future AQI values
    def forecast_future(model, data, seq_length, future_steps):
        future_predictions = []
        current_seq = data[-seq_length:]  # Start from the last available sequence
        
        for _ in range(future_steps):
            pred = model.predict(current_seq[np.newaxis, :, :])  # Predict next step
            future_predictions.append(pred[0])
            current_seq = np.append(current_seq[1:], pred, axis=0)  # Update the sequence
        
        return np.array(future_predictions)

    # Load pre-trained models and forecast the next 'n' hours
    future_predictions = {}
    for col in aqi_columns:
        model = load_model(f'{col}_LSTM_model.h5')  # Load pre-trained LSTM model
        future_scaled_predictions = forecast_future(model, scaled_data[col], seq_length, future_steps)
        # Inverse transform to get the actual AQI values
        future_predictions[col] = scalers[col].inverse_transform(future_scaled_predictions).flatten()

    # Convert future predictions to a DataFrame with a proper timestamp index
    future_df = pd.DataFrame(future_predictions, index=pd.date_range(start=df.index[-1], periods=future_steps+1, freq='h')[1:])
    return future_df

# File path of the dataset
file_path = '../ARIF/AQI_Weather_Data.csv'

# Call the forecast function to get future AQI predictions
future_df = forecast_aqi(file_path)

# Display the predicted values
print(future_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
                     AQI_PM2.5    AQI_PM10    AQI_NO2     AQI_CO    AQI_SO2
2023-04-01 00:00:00  42.472965  102.229370  49.327312  31.932623  28.024145
2023-04-01 01:00:00  43.372379  102.931725  49.198601  33.566483  28.203175
2023-04-01 02:00:00  44.211086  103.279778  49.051525  34.603947  28.438957
2023-04-01 03:00:00  44.921944  103.427887  48.899712  35.322994  28.714672
2023-04-01 04:00:00  45.542187  103.380775  48.747456  36.013939  29.024403
2023-04-01 05:00:00  46.092628  103.171753  48.596684  36.850441  29.361322
2023-04-01 06:00:00  46.587616  102.838280  48.447876  37.833908  29.719286
2023-04-01 07:00:00  47.038719  102.413216  48.300694  38.829639  30.093285


In [10]:
for row in future_df.iterrows():
    print(row[1])
    print(type(row[1]))


AQI_PM2.5     42.472965
AQI_PM10     102.229370
AQI_NO2       49.327312
AQI_CO        31.932623
AQI_SO2       28.024145
Name: 2023-04-01 00:00:00, dtype: float32
<class 'pandas.core.series.Series'>
AQI_PM2.5     43.372379
AQI_PM10     102.931725
AQI_NO2       49.198601
AQI_CO        33.566483
AQI_SO2       28.203175
Name: 2023-04-01 01:00:00, dtype: float32
<class 'pandas.core.series.Series'>
AQI_PM2.5     44.211086
AQI_PM10     103.279778
AQI_NO2       49.051525
AQI_CO        34.603947
AQI_SO2       28.438957
Name: 2023-04-01 02:00:00, dtype: float32
<class 'pandas.core.series.Series'>
AQI_PM2.5     44.921944
AQI_PM10     103.427887
AQI_NO2       48.899712
AQI_CO        35.322994
AQI_SO2       28.714672
Name: 2023-04-01 03:00:00, dtype: float32
<class 'pandas.core.series.Series'>
AQI_PM2.5     45.542187
AQI_PM10     103.380775
AQI_NO2       48.747456
AQI_CO        36.013939
AQI_SO2       29.024403
Name: 2023-04-01 04:00:00, dtype: float32
<class 'pandas.core.series.Series'>
AQI_PM2.5 

In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import pandas as pd

def forecast_aqi_from_current(current_aqi, seq_length=24, future_steps=8):
    """
    This function forecasts future AQI values based on the current AQI data passed as input.

    :param current_aqi: A dictionary with current AQI values, e.g.:
                        {'AQI_PM2.5': 45.0, 'AQI_PM10': 60.0, 'AQI_NO2': 20.0, 'AQI_CO': 1.5, 'AQI_SO2': 10.0}
    :param seq_length: Number of previous steps to use for the LSTM model (default: 24 hours)
    :param future_steps: Number of future hours to predict (default: 8 hours)
    :return: A DataFrame with the predicted future AQI values for the next 'future_steps' hours.
    """
    
    aqi_columns = ['AQI_PM2.5', 'AQI_PM10', 'AQI_NO2', 'AQI_CO', 'AQI_SO2']

    # Normalize current AQI values
    scalers = {}
    scaled_current_aqi = {}
    
    for col in aqi_columns:
        scaler = MinMaxScaler()
        # Fit the scaler on a range, assuming typical AQI value ranges (e.g., 0 to 500 for PM2.5 and PM10, etc.)
        scaler.fit(np.array([0, 500]).reshape(-1, 1))
        scaled_current_aqi[col] = scaler.transform(np.array(current_aqi[col]).reshape(-1, 1))
        scalers[col] = scaler

    # Function to forecast future AQI values
    def forecast_future(model, data, seq_length, future_steps):
        future_predictions = []
        current_seq = np.array(data[-seq_length:])  # Start from the last available sequence
        
        for _ in range(future_steps):
            pred = model.predict(current_seq[np.newaxis, :, :])  # Predict next step
            future_predictions.append(pred[0])
            current_seq = np.append(current_seq[1:], pred, axis=0)  # Update the sequence

        return np.array(future_predictions)

    # Load pre-trained models and forecast the next 'n' hours
    future_predictions = {}
    
    for col in aqi_columns:
        model = load_model(f'{col}_LSTM_model.h5')  # Load pre-trained LSTM model
        
        # Assuming current AQI values serve as the last known sequence
        future_scaled_predictions = forecast_future(model, scaled_current_aqi[col], seq_length, future_steps)
        
        # Inverse transform to get the actual AQI values
        future_predictions[col] = scalers[col].inverse_transform(future_scaled_predictions).flatten()

    # Convert future predictions to a DataFrame with a proper timestamp index
    future_df = pd.DataFrame(future_predictions, index=pd.date_range(start=pd.Timestamp.now(), periods=future_steps, freq='H'))
    return future_df

# Example Usage:
current_aqi = {
    'AQI_PM2.5': 45.0,
    'AQI_PM10': 60.0,
    'AQI_NO2': 20.0,
    'AQI_CO': 1.5,
    'AQI_SO2': 10.0
}

# Call the function to get future AQI predictions
future_df = forecast_aqi_from_current(current_aqi)

# Display the predicted future AQI values
print(future_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
                             AQI_PM2.5    AQI_PM10    AQI_NO2      AQI_CO  \
2024-08-19 21:52:35.250874  -18.761978   33.683098  23.624792  -45.963631   
2024-08-19 22:52:35.250874 -110.532150   -1.685523  26.746155 -103.369652   
2024-08-19 23:52:35.250874 -215.434692  -46.177261  29.455200 -166.950134   
2024-08-20 00:52:35.250874 -302.007690  -97.323051  31.822323 -230.052765   
2024-08-20 01:52:35.250874 -352.337891 -149.779343  33.902912 -285.457611   
2024-08-20 02:52:35.250874 -374.706970 -196.995850  35.741009 -328.595825   
2024-08-20 03:52:35.250874 -383.282349 -234.247513  37.372246 -358.900909   
2024-08-20 04:52:35.250874 -386.366547 -260.417633  38.825699 -378.597504   

         

/var/folders/0b/tbcb989n38q5j2gz3d4zcx680000gn/T/ipykernel_93210/3387477458.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_df = pd.DataFrame(future_predictions, index=pd.date_range(start=pd.Timestamp.now(), periods=future_steps, freq='H'))


In [3]:
from aqi_cal import calculate_aqi, get_aqi_cat

breakpoints = {
    'PM2.5': [(0, 30, 0, 50), (31, 60, 51, 100), (61, 90, 101, 200), (91, 120, 201, 300), (121, 250, 301, 400), (251, 500, 401, 500)],
    'PM10': [(0, 50, 0, 50), (51, 100, 51, 100), (101, 250, 101, 200), (251, 350, 201, 300), (351, 430, 301, 400), (431, 500, 401, 500)],
    'NO2': [(0, 40, 0, 50), (41, 80, 51, 100), (81, 180, 101, 200), (181, 280, 201, 300), (281, 400, 301, 400), (401, 1000, 401, 500)],
    'SO2': [(0, 40, 0, 50), (41, 80, 51, 100), (81, 380, 101, 200), (381, 800, 201, 300), (801, 1600, 301, 400), (1601, 2100, 401, 500)],
    'CO': [(0, 1, 0, 50), (1.1, 2, 51, 100), (2.1, 10, 101, 200), (10.1, 17, 201, 300), (17.1, 34, 301, 400), (34.1, 50, 401, 500)],
}

def ret(pm2_5, pm10, no2, co, so2):
    try:
        # Dictionary to store provided pollutant values
        data = {
            "PM2.5": pm2_5,
            "PM10": pm10,
            "NO2": no2,
            "CO": co,
            "SO2": so2
        }

        # Calculate AQI for each relevant pollutant based on its concentration and breakpoints
        aqi_val = {}
        for pollutant, concentration in data.items():
            if pollutant in ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2']:
                aqi_val[f'{pollutant}'] = calculate_aqi(concentration, breakpoints[pollutant])

        # Calculate overall AQI by selecting the maximum value from calculated AQI values
        overall_aqi = max(aqi_val.values())

        # Get the AQI category and health impact based on the overall AQI
        remark, health_impact = get_aqi_cat(overall_aqi)

        # Identify the pollutant that contributed the most to the AQI
        pollutant_res = list(filter(lambda x: aqi_val[x] == overall_aqi, aqi_val))[0]

        # Return the final result
        return {
            "aqi": overall_aqi,
            "remark": remark,
            "impact": health_impact,
            "pollutant_res": pollutant_res
        }

    except Exception as e:
        print(f"Exception in ret method: {e}")
        return str(e)


In [4]:
future_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8 entries, 2023-04-01 00:00:00 to 2023-04-01 07:00:00
Freq: h
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AQI_PM2.5  8 non-null      float32
 1   AQI_PM10   8 non-null      float32
 2   AQI_NO2    8 non-null      float32
 3   AQI_CO     8 non-null      float32
 4   AQI_SO2    8 non-null      float32
dtypes: float32(5)
memory usage: 224.0 bytes


In [8]:
future_df["AQI_CO"] /= 1000
for row in future_df.iterrows():
    print(type(row[1][1]))
    print(ret(row[1]["AQI_PM2.5"], row[1]["AQI_PM10"], row[1]["AQI_NO2"], row[1]["AQI_CO"], row[1]["AQI_SO2"]))

<class 'numpy.float32'>
{'aqi': 101.817, 'remark': 'Moderate', 'impact': 'Breathing discomfort to the people with lungs, asthma, and heart diseases', 'pollutant_res': 'PM10'}
<class 'numpy.float32'>
{'aqi': 102.283, 'remark': 'Moderate', 'impact': 'Breathing discomfort to the people with lungs, asthma, and heart diseases', 'pollutant_res': 'PM10'}
<class 'numpy.float32'>
{'aqi': 102.515, 'remark': 'Moderate', 'impact': 'Breathing discomfort to the people with lungs, asthma, and heart diseases', 'pollutant_res': 'PM10'}
<class 'numpy.float32'>
{'aqi': 102.613, 'remark': 'Moderate', 'impact': 'Breathing discomfort to the people with lungs, asthma, and heart diseases', 'pollutant_res': 'PM10'}
<class 'numpy.float32'>
{'aqi': 102.582, 'remark': 'Moderate', 'impact': 'Breathing discomfort to the people with lungs, asthma, and heart diseases', 'pollutant_res': 'PM10'}
<class 'numpy.float32'>
{'aqi': 102.443, 'remark': 'Moderate', 'impact': 'Breathing discomfort to the people with lungs, asth

/var/folders/0b/tbcb989n38q5j2gz3d4zcx680000gn/T/ipykernel_19500/4190414085.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(type(row[1][1]))


In [25]:
import sys
print(sys.executable)

/usr/local/bin/python3
